In [ ]:
# default_exp core

# pytrim2

> A python program for trimming and demultiplexing naopore reads.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
# Import dependencies
from Bio import SeqIO
import numpy as np
from Bio import Align
from Bio import pairwise2

In [ ]:
#export

def findAdapter(seq_record, barcode_primer, cutOff):
    # Alignment using pairwise2
    alignments = pairwise2.align.localms(seq_record.seq, barcode_primer, 1, 0, -2, -2)

    # Find best match
    alingmentAccuracy = np.array( [i.score for i in alignments]  )
    alingmentEnd = np.array( [i.end for i in alignments]  )
    maxAcc = np.max(alingmentAccuracy)
    maxAcc = np.max(alingmentAccuracy)
    maxAccAlignment = np.where(alingmentAccuracy == maxAcc)[0]
    inwardAlingmentEnd = np.max(alingmentEnd[maxAccAlignment])
    
    maxAcc = maxAcc / len(barcode_primer) * 100

    if maxAcc >= cutOff:
        print("TRUE")
    maxAcc = maxAcc
    print("queried length:", len(seq_record.seq), "; accuracy [%]:", maxAcc)

In [ ]:
#export

def findAlingments(record_dict, barcode_primer, inward_end):
    
    record_keys = list(record_dict.keys())
    
    aligner = Align.PairwiseAligner()
    aligner.match_score = 1.0
    aligner.mismatch_score = 0
    aligner.gap_score = -2
    aligner.mode = "local"
    
    al = "none"
    for i in record_keys:
        al_array = np.zeros(10)
        seq = record_dict[i].seq[0:200]        
        alignments = aligner.align(seq, barcode_primer)
        len_alignments = len(alignments)
        if(len_alignments <= 10):
            score = alignments.score
            al = [i.aligned for i in alignments]
            for i in [1,2,3]:
                al = [x for xs in al for x in xs]
            
    return(al)
    
    

In [ ]:
#slow

record_dict = SeqIO.index("test_data/test.fastq", "fastq")
barcode_primer = "AATGCGAAG"
findAlingments(record_dict, barcode_primer, 200)

[983, 991, 0, 8, 1151, 1160, 0, 9]

In [ ]:
record_keys = list(record_dict.keys())
    
aligner = Align.PairwiseAligner()
aligner.match_score = 1.0
aligner.mismatch_score = 0
aligner.gap_score = -2
aligner.mode = "local"
    
al = "none"
for i in record_keys[0:100]:
    al_array = np.zeros(10)
    seq = record_dict[i].seq[0:200]        
    alignments = aligner.align(seq, barcode_primer)
    len_alignments = len(alignments)
    if(len_alignments <= 10):
        score = alignments.score
        al = [i.aligned for i in alignments]
        for i in [1,2,3]:
            al = [x for xs in al for x in xs]
        

print(al)



[93, 101, 0, 8, 117, 125, 0, 8]
